In [29]:
# nltk.download('stopwords')

In [30]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [31]:
# Gensim
from gensim.utils import simple_preprocess
from gensim.models import LdaModel, Phrases
from gensim.models.phrases import Phraser
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [32]:
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
from nltk.corpus import stopwords
from nltk import Tree

stop_words = stopwords.words('english')

In [34]:
df = pd.read_csv("usnewspaper_china.csv", header=None,)

In [35]:
data = df[0].dropna().tolist()

In [36]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

<>:2: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
/var/folders/fd/qmhdtpdj7kj77ftrqfj87hmc0000gn/T/ipykernel_70879/1419459543.py:2: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
/var/folders/fd/qmhdtpdj7kj77ftrqfj87hmc0000gn/T/ipykernel_70879/1419459543.py:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [37]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[0])

['in', 'the', 'new', 'sino', 'russian', 'defense', 'relationship', 'china', 'does', 'what', 'it', 'wants', 'and', 'there', 'isn', 'whole', 'lot', 'russia', 'can', 'do', 'about', 'it', 'xi', 'jingping', 'recent', 'visit', 'to', 'moscow', 'his', 'first', 'since', 'vladimir', 'putin', 'invaded', 'ukraine', 'last', 'year', 'was', 'summed', 'up', 'by', 'historian', 'sergey', 'radchenko', 'in', 'this', 'way', 'the', 'summit', 'can', 'be', 'summarised', 'by', 'the', 'chinese', 'saying', '雷声大雨点小', 'loud', 'thunder', 'but', 'few', 'raindrops', 'scratch', 'that', 'even', 'the', 'thunder', 'wasn', 'all', 'that', 'loud', 'the', 'meeting', 'which', 'apparently', 'produced', 'no', 'major', 'policy', 'shifts', 'nor', 'even', 'notable', 'statements', 'of', 'support', 'did', 'further', 'illustrate', 'tectonic', 'shift', 'in', 'the', 'supposed', 'no', 'limits', 'relationship', 'china', 'is', 'taking', 'the', 'lead', 'in', 'nearly', 'every', 'aspect', 'including', 'in', 'the', 'defense', 'industrial', 's

In [38]:
# Define functions for stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops)

In [40]:
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

## Bigrams and Trigrams

In [41]:
bigram = Phrases(data_words, min_count=20, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [42]:
print("Specific Bigram Example:", ' '.join(bigram_mod[data_words[1]]))
                                                      
print("Specific Trigram Example:", ' '.join(trigram_mod[bigram_mod[data_words[0]]]))

Specific Bigram Example: cookie_preferences cookie_list cookie_list cookie is small_piece of data text_file that website when visited by user_asks your_browser to store on your device in order to remember information about you such as your language_preference or login information those cookies are set by us and called first party cookies we also use third party cookies which are cookies from domain_different than the domain of the website you are visiting for our advertising and marketing efforts more specifically we use cookies and other tracking technologies for the following purposes_strictly necessary cookies we do not allow you to opt_out of our certain_cookies as they are necessary to ensure the proper_functioning of our website such as prompting our cookie_banner and remembering_your privacy_choices and or to monitor_site performance these cookies are not used in way that constitutes_sale of your data under the ccpa_you can set your_browser to block or alert you about these cook

In [43]:
data_with_phrases = [trigram_mod[bigram_mod[doc]] for doc in data_words]

In [72]:
data_with_phrases[1][:30]

['cookie_preferences_cookie_list',
 'cookie_list_cookie',
 'is',
 'small_piece',
 'of',
 'data',
 'text_file',
 'that',
 'website',
 'when',
 'visited',
 'by',
 'user_asks_your_browser',
 'to',
 'store',
 'on',
 'your',
 'device',
 'in',
 'order',
 'to',
 'remember',
 'information',
 'about',
 'you',
 'such',
 'as',
 'your_language_preference',
 'or',
 'login_information']

In [74]:
# Create Dictionary
id2word = corpora.Dictionary(data_with_phrases)
id2word.filter_extremes(no_below=20, no_above=0.5) # brings down unique tokens from 80 thousand to 12


# Create Corpus
texts = data_with_phrases

# Term Frequency list   
corpus = [id2word.doc2bow(text) for text in texts] 

In [77]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [75]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                id2word=id2word,
                num_topics=20, 
                random_state=100,
                update_every=1,
                chunksize=100,
                passes=10,
                alpha='auto',
                per_word_topics=True)

In [76]:
print('Number of unique tokens: %d' % len(id2word))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 12495
Number of documents: 10000


In [78]:
# Check the type of the numbers in the LDA topics
for topic in lda_model.print_topics():
    for word, prob in lda_model.show_topic(topic[0]):
        if isinstance(prob, complex):
            print(f"Complex number found in topic {topic[0]} for word '{word}'")

2023-11-04 20:27:06,920 : INFO : topic #0 (0.030): 0.100*"die" + 0.059*"der" + 0.054*"und" + 0.032*"das" + 0.029*"von" + 0.023*"zu" + 0.020*"ist" + 0.020*"fur" + 0.018*"da" + 0.017*"den"
2023-11-04 20:27:06,924 : INFO : topic #1 (0.056): 0.045*"release" + 0.041*"institute" + 0.039*"found" + 0.035*"were" + 0.035*"archaeology" + 0.028*"academy" + 0.028*"news" + 0.023*"researchers" + 0.021*"photo" + 0.021*"according"
2023-11-04 20:27:06,930 : INFO : topic #2 (0.188): 0.019*"economic" + 0.015*"economy" + 0.013*"country" + 0.011*"government" + 0.010*"than" + 0.010*"world" + 0.010*"growth" + 0.009*"their" + 0.009*"countries" + 0.008*"they"
2023-11-04 20:27:06,931 : INFO : topic #3 (0.359): 0.023*"he" + 0.017*"we" + 0.013*"our" + 0.012*"biden" + 0.012*"american" + 0.011*"his" + 0.009*"us" + 0.009*"about" + 0.009*"president" + 0.007*"who"
2023-11-04 20:27:06,932 : INFO : topic #4 (0.105): 0.023*"growth" + 0.022*"you" + 0.019*"our" + 0.018*"data" + 0.016*"economy" + 0.014*"we" + 0.013*"cookies"

In [79]:
# Print the Keyword
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

2023-11-04 20:27:25,614 : INFO : topic #0 (0.030): 0.100*"die" + 0.059*"der" + 0.054*"und" + 0.032*"das" + 0.029*"von" + 0.023*"zu" + 0.020*"ist" + 0.020*"fur" + 0.018*"da" + 0.017*"den"
2023-11-04 20:27:25,621 : INFO : topic #1 (0.056): 0.045*"release" + 0.041*"institute" + 0.039*"found" + 0.035*"were" + 0.035*"archaeology" + 0.028*"academy" + 0.028*"news" + 0.023*"researchers" + 0.021*"photo" + 0.021*"according"
2023-11-04 20:27:25,623 : INFO : topic #2 (0.188): 0.019*"economic" + 0.015*"economy" + 0.013*"country" + 0.011*"government" + 0.010*"than" + 0.010*"world" + 0.010*"growth" + 0.009*"their" + 0.009*"countries" + 0.008*"they"
2023-11-04 20:27:25,624 : INFO : topic #3 (0.359): 0.023*"he" + 0.017*"we" + 0.013*"our" + 0.012*"biden" + 0.012*"american" + 0.011*"his" + 0.009*"us" + 0.009*"about" + 0.009*"president" + 0.007*"who"
2023-11-04 20:27:25,631 : INFO : topic #4 (0.105): 0.023*"growth" + 0.022*"you" + 0.019*"our" + 0.018*"data" + 0.016*"economy" + 0.014*"we" + 0.013*"cookies"

[(0, '0.100*"die" + 0.059*"der" + 0.054*"und" + 0.032*"das" + 0.029*"von" + 0.023*"zu" + 0.020*"ist" + 0.020*"fur" + 0.018*"da" + 0.017*"den"'), (1, '0.045*"release" + 0.041*"institute" + 0.039*"found" + 0.035*"were" + 0.035*"archaeology" + 0.028*"academy" + 0.028*"news" + 0.023*"researchers" + 0.021*"photo" + 0.021*"according"'), (2, '0.019*"economic" + 0.015*"economy" + 0.013*"country" + 0.011*"government" + 0.010*"than" + 0.010*"world" + 0.010*"growth" + 0.009*"their" + 0.009*"countries" + 0.008*"they"'), (3, '0.023*"he" + 0.017*"we" + 0.013*"our" + 0.012*"biden" + 0.012*"american" + 0.011*"his" + 0.009*"us" + 0.009*"about" + 0.009*"president" + 0.007*"who"'), (4, '0.023*"growth" + 0.022*"you" + 0.019*"our" + 0.018*"data" + 0.016*"economy" + 0.014*"we" + 0.013*"cookies" + 0.012*"may" + 0.011*"year" + 0.010*"or"'), (5, '0.163*"de" + 0.065*"la" + 0.059*"en" + 0.055*"el" + 0.054*"que" + 0.021*"los" + 0.017*"un" + 0.017*"para" + 0.016*"una" + 0.015*"no"'), (6, '0.020*"covid" + 0.013*"pe

In [47]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [80]:
try:
    vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
except Exception as e:
    print(f"Error preparing LDA visualization: {e}")

In [81]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.115400  0.018500       1        1  12.577384
3     -0.132272  0.129424       2        1  10.090392
19    -0.157090  0.117657       3        1   9.304742
18    -0.117348  0.139303       4        1   7.257911
2     -0.136391 -0.022398       5        1   6.656205
8     -0.119149  0.018166       6        1   6.437281
16    -0.105546  0.121764       7        1   6.382124
6     -0.130778 -0.074216       8        1   6.221630
13    -0.070983 -0.168095       9        1   5.877481
14    -0.124673  0.084519      10        1   5.609887
7     -0.069974  0.054145      11        1   5.491495
5      0.311870  0.178360      12        1   5.008658
10    -0.063807 -0.135753      13        1   3.841718
4      0.038425 -0.167847      14        1   2.606747
12     0.054439 -0.124254      15        1   1.863326
11     0.079593 -0.208860      16        1   1.366662
1      0.034477 -0.043961      17        1   1.067881
0      0.334726  0.002203      18        1   0.982811
17     0.266748  0.158466      19        1   0.737842
9      0.223132 -0.077122      20        1   0.617821, topic_info=               Term          Freq         Total Category  logprob  loglift
688              de  25511.000000  25511.000000  Default  30.0000  30.0000
552              us  12155.000000  12155.000000  Default  29.0000  29.0000
748              la  10000.000000  10000.000000  Default  28.0000  28.0000
707              en   9140.000000   9140.000000  Default  27.0000  27.0000
704              el   8553.000000   8553.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
1122         latest    166.722295   1565.405190  Topic20  -4.7413   2.8472
258          market    191.380228   5223.734807  Topic20  -4.6033   1.7800
641           south    172.680197   4087.083571  Topic20  -4.7062   1.9226
2132         member    144.269608    863.757105  Topic20  -4.8859   3.2971
213   international    154.510412   5538.389828  Topic20  -4.8173   1.5075

[1330 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
0         1  0.736969               able
0         2  0.022070               able
0         3  0.011823               able
0         8  0.031528               able
0        10  0.107984               able
...     ...       ...                ...
2015      8  0.997312  zero_covid_policy
2248      6  0.067527              zhang
2248      8  0.905289              zhang
2248     12  0.027433              zhang
5049     18  0.998794                 zu

[3234 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 4, 20, 19, 3, 9, 17, 7, 14, 15, 8, 6, 11, 5, 13, 12, 2, 1, 18, 10])

In [84]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

2023-11-04 20:35:44,066 : INFO : -10.303 per-word bound, 1263.1 perplexity estimate based on a held-out corpus of 10000 documents with 3091994 words



Perplexity:  -10.302702333938628


In [86]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_with_phrases, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

2023-11-04 20:35:44,232 : INFO : using ParallelWordOccurrenceAccumulator(processes=9, batch_size=64) to estimate probabilities from sliding windows
2023-11-04 20:35:50,152 : INFO : 1 batches submitted to accumulate stats from 64 documents (24925 virtual)
2023-11-04 20:35:50,157 : INFO : 2 batches submitted to accumulate stats from 128 documents (48294 virtual)
2023-11-04 20:35:50,163 : INFO : 3 batches submitted to accumulate stats from 192 documents (72845 virtual)
2023-11-04 20:35:50,168 : INFO : 4 batches submitted to accumulate stats from 256 documents (100699 virtual)
2023-11-04 20:35:50,174 : INFO : 5 batches submitted to accumulate stats from 320 documents (124366 virtual)
2023-11-04 20:35:50,183 : INFO : 6 batches submitted to accumulate stats from 384 documents (159291 virtual)
2023-11-04 20:35:50,187 : INFO : 7 batches submitted to accumulate stats from 448 documents (178018 virtual)
2023-11-04 20:35:50,202 : INFO : 8 batches submitted to accumulate stats from 512 documents (


Coherence Score:  0.5655869210194248


# Create tree representation

In [ ]:
data_lemmatized[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [ ]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [ ]:
doc = nlp(data[0])
[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [85]:
from IPython.display import HTML
css_str = '<style> \
.jp-Button path { fill: #616161;} \
text.terms { fill: #616161;} \
.jp-icon-warn0 path {fill: var(--jp-warn-color0);} \
.bp3-button-text path { fill: var(--jp-inverse-layout-color3);} \
.jp-icon-brand0 path { fill: var(--jp-brand-color0);} \
text.terms { fill: #616161;} \
</style>'
display(HTML(css_str ))